In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,regexp_replace
import pyarrow as pa
import pyarrow.parquet as pq

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::172684736408:role/LabRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 86e34a21-3d31-4251-8ab1-7434c055d74b
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
Waiting for session 86e34a21-3d31-4251-8ab1-7434c055d74b to get into ready status...
Session 86e34a21-3d31-4251-8ab1-7434c055d74b has been created.



In [2]:
# Inicialize a sessão Spark
spark = SparkSession.builder \
    .appName("ReadS3DataJoin") \
    .getOrCreate()

input_s3_path_bancos = "s3://172684736408-trusted/Bancos/*.parquet"
input_s3_path_empregados = "s3://172684736408-trusted/Empregados/*.parquet"
input_s3_path_reclamacoes = "s3://172684736408-trusted/Reclamações/*.parquet"

output_s3_path_delivery = "s3://172684736408-delivery/bancos_satisfacao"

df_bancos = spark.read.option("encoding", "UTF-8").parquet(input_s3_path_bancos)
df_empregados = spark.read.option("encoding", "UTF-8").parquet(input_s3_path_empregados)
df_reclamacoes = spark.read.option("encoding", "UTF-8").parquet(input_s3_path_reclamacoes)

In [3]:
df_join_reclama_empreg = df_empregados.join(df_reclamacoes, on="Nome", how="inner")
final_df = df_join_reclama_empreg.join(df_bancos, on="Nome", how="inner")

In [4]:
final_df.write.format("parquet").mode("overwrite").option("path", f"{output_s3_path_delivery}").save()